[View in Colaboratory](https://colab.research.google.com/github/paxx13/gym/blob/master/DP_MC_for_MDF.ipynb)

this notebook shows dynamic programming techniques to solve a markov devision process(MDP) as well as model free methods.

In [0]:
%%capture
!pip install gym

In [0]:
from __future__ import (absolute_import, division, print_function,
unicode_literals)
import gym
import numpy as np
import time
from random import randint

define the environment

In [0]:
env = gym.make('FrozenLake-v0')
env.reset()
env.render()
'''
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
'''

random search

In [0]:
def run_random_policy(env):
  observation = env.reset()
  total_reward = 0
  num_steps = 0
  for i in range(100):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)

    total_reward += reward
    num_steps += 1

    if done:
      print('===========  policy done ==========')
      break
      
  return num_steps, total_reward

In [0]:
best_params = None
best_reward = 0

for _ in range(1000):
  steps, reward  = run_random_policy(env)
  if reward > best_reward:
    best_reward = reward
    best_steps = steps
    if reward == 1:
      break
      
print('best reward: ', best_reward)
print('number of steps taken: ', best_steps)

# Dynamic Programming

## policy iteration

In [0]:
def policy_evaluation(env, policy, gamma, e=1e-10):
  v = np.zeros(env.env.nS)
  while True:
    v_p = np.copy(v)
    for s in range(env.env.nS):
      a = policy[s]
      v[s] = sum([p * (r + gamma*v_p[s_]) for p, s_, r, _ in env.env.P[s][a]])
    if (np.sum(np.fabs(v_p-v)) <= e):
      break;
  return v

In [0]:
def policy_improvement(env, v, gamma):
  policy = np.zeros(env.env.nS) 
  for s in range(env.env.nS):
    q_sa = np.zeros(env.env.nA)
    for a in range(env.env.nA):
      for p, s_, r, _ in env.env.P[s][a]:
        q_sa[a] = np.sum(p * (r + gamma*v[s_]))
    policy[s] = np.argmax(q_sa)
  return policy

In [0]:
def policy_iteration(env, gamma):
  policy = np.random.choice(env.env.nA, size=(env.env.nS))
  for i in range(1000):
    v      = policy_evaluation(env, policy, gamma)
    policy_= policy_improvement(env, v, gamma)

    if (np.all(policy == policy_)):
      break

    policy = policy_  
  return policy, v 

In [0]:
gamma = 1.0

start_time = time.time()
policy, v  = policy_iteration(env, gamma)
end_time = time.time()

print('optimal value function:\n', v.reshape(4,4))
print('------------------------')
print('optimal policy:\n', policy.reshape(4,4))  
print('------------------------')
print('time:\n', end_time - start_time)  

## value iteration

In [0]:
def value_iteration(env, gamma, e=1e-8):
  v = np.zeros(env.env.nS)
  policy = np.zeros(env.env.nS) 
  q_sa = np.zeros(env.env.nA)
  
  for i in range(10000):
    v_p = np.copy(v)
    for s in range(env.env.nS):
      for a in range(env.env.nA):
        q_sa[a] = sum([p * (r + gamma*v_p[s_]) for p, s_, r, _ in env.env.P[s][a]])
      v[s] = max(q_sa)
    if (np.sum(np.fabs(v_p-v)) <= e):
      break;
  
  for s in range(env.env.nS):
    for a in range(env.env.nA):
        q_sa[a] = sum([p * (r + gamma*v[s_]) for p, s_, r, _ in env.env.P[s][a]])
    policy[s] = np.argmax(q_sa)
  
  return policy, v

In [0]:
gamma = 1.0

start_time = time.time()
policy, v  = value_iteration(env, gamma)
end_time = time.time()

print('optimal value function:\n', v.reshape(4,4))
print('------------------------')
print('optimal policy:\n', policy.reshape(4,4))  
print('------------------------')
print('time:\n', end_time - start_time)  

# Model Free Control

In [0]:
def show_progress(e, r,  n=1000):
  if e % n == 0:
    print('Episode = ', e, ' | Avg Reward = ', r/n )

In [0]:
def get_policy_from_q(env, q):
  p = np.empty(env.env.nS)
  for s in range(env.env.nS):
    p[s] = (np.argmax(q[s]))
  return p    

## Monte Carlo

In [0]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [0]:
def generate_episode(env, policy, q):
    done = False
    s = env.reset()
    episode= []
    while not done:
      #probs = policy(s)   
      #action = np.random.choice(np.arange(len(probs)), p=probs)
      action = epsilon_greedy_policy(env, q, s)
      s1 , reward, done, info = env.step(action)
      episode.append((s, action, reward))
      s = s1
        
    return episode

In [0]:
from collections import defaultdict
  
def mc_control(env, gamma, epsilon=0.1, episodes=10000): 
  q= defaultdict(lambda: np.zeros(env.action_space.n))
  policy = make_epsilon_greedy_policy(q, epsilon, env.action_space.n)
  returns = defaultdict(float)
  returns_count = defaultdict(float)

  for e in range(episodes):
    G = {} # return on state
    C = 0 # cumulative reward
    episode = generate_episode(env, policy, q)
    avg_reward = 0#np.average(episode[2])
    for (state, action, reward) in reversed(episode):
      G[(state, action)] = C = reward + gamma*C
    for sa in G:
      state, action = sa
      returns[sa] += G[sa]
      returns_count[sa] += 1
      q[state][action] = returns[sa] / returns_count[sa]
    show_progress(e, avg_reward)
  return q

In [0]:
gamma = 0.99

start_time = time.time()
q  = mc_control(env, gamma, episodes=100000)
end_time = time.time()

p = get_policy_from_q(env, q)
  
print('optimal policy:\n', p.reshape(4,4))
print('------------------------')
print('time:\n', end_time - start_time)  

## Q Learning

In [0]:
def epsilon_greedy_policy(env, q, s, epsilon=0.1):
  if np.random.uniform(0, 1) < epsilon:
    #explore
    action = env.action_space.sample() 
  else:
    #exploit
    action = np.argmax(q[s]) # Exploit learned values
    
  return action

In [0]:
def q_learning(env, alpha, gamma, epsilon=0.1, episodes=10000):
  q = np.zeros([env.env.nS, env.env.nA])
  
  for e in range(episodes):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
      action = epsilon_greedy_policy(env, q, state)
      s1, reward, done, _ = env.step(action) 
      
      old_value = q[state, action]
      next_max = np.max(q[s1])
        
      new_value = (1 - alpha) * old_value + alpha * (reward + gamma * s1)
      q[state, action] = new_value
      
      state = s1
      
    show_progress(e, 0)
    
  return q

In [0]:
# Hyperparameters
alpha = 0.1
gamma = 0.9
epsilon = 0.1

start_time = time.time()
q = q_learning(env, alpha, gamma, episodes=500000)
end_time = time.time()

p = get_policy_from_q(env, q)
print('optimal policy:\n', p.reshape(4,4))
print('------------------------')
print('time:\n', end_time - start_time)  